In [41]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import math
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
# Load pylab
%pylab inline

destinations = pd.read_csv("data/my_submission.csv")
test_data = pd.read_csv("data/test.csv",header=0)
train_data = pd.read_csv("data/train.csv",header=0)
firstColumnInd = destinations["Id"]
train_data['weekday'] = pd.DatetimeIndex(train_data['dteday']).dayofweek

Populating the interactive namespace from numpy and matplotlib


/Users/timothee001/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['clf', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [42]:
train_data.head(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,2011-01-01,1,0,1,0,0,5,0,1,0.24,0.2879,0.81,0.0,16
1,2,2011-01-01,1,0,1,1,0,5,0,1,0.22,0.2727,0.80,0.0,40
2,3,2011-01-01,1,0,1,2,0,5,0,1,0.22,0.2727,0.80,0.0,32
3,4,2011-01-01,1,0,1,3,0,5,0,1,0.24,0.2879,0.75,0.0,13
4,5,2011-01-01,1,0,1,4,0,5,0,1,0.24,0.2879,0.75,0.0,1


In [43]:
feature_columns_to_use = ['season','hr','yr','mnth','weekday','holiday','workingday','weathersit','atemp','hum','windspeed']

X_train = train_data[feature_columns_to_use]
y_train = train_data['cnt']


In [44]:
X_train.head(4)

,season,hr,yr,mnth,weekday,holiday,workingday,weathersit,atemp,hum,windspeed
0,1,0,0,1,5,0,0,1,0.2879,0.81,0.0
1,1,1,0,1,5,0,0,1,0.2727,0.80,0.0
2,1,2,0,1,5,0,0,1,0.2727,0.80,0.0
3,1,3,0,1,5,0,0,1,0.2879,0.75,0.0


In [45]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

X = X_train
y = y_train

X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.2, random_state=0)

tuned_parameters = [{'n_estimators':[10,100,500]}]   
    
scores = ['r2']

for score in scores:
    
    print score
    
    clf = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=5, scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print ""
    #best_estimator_ returns the best estimator chosen by the search
    print(clf.best_estimator_)
    print ""
    print("Grid scores on development set:")
    print ""
    #grid_scores_ returns:
    #    * a dict of parameter settings
    #    * the mean score over the cross-validation folds 
    #    * the list of scores for each fold
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print ""
        

r2
Best parameters set found on development set:

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

Grid scores on development set:

0.938 (+/-0.003) for {'n_estimators': 10}
0.947 (+/-0.003) for {'n_estimators': 100}
0.947 (+/-0.003) for {'n_estimators': 500}



In [46]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(X_train, y_train)

In [47]:
predictions = forest.predict(test_data[feature_columns_to_use])
print predictions

[  5   5   5 ..., 106 106  87]


In [48]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'Id': test['instant'],
                            'Prediction': predictions })
submission = submission[['Id','Prediction']]

submission.to_csv("submission.csv", index=False)

TypeError: 'instancemethod' object has no attribute '__getitem__'